<a href="https://colab.research.google.com/github/aosman96/Predict-Future-Sales-using-FbProphet-with-high-speed-and-new-Algorthim/blob/master/Predict_Future_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from fbprophet import *
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist, pdist
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive/')

%cd gdrive/My Drive/BigData
%pwd
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/
/content/gdrive/My Drive/BigData
BigData.drawio       sample_submission.csv       test.csv
item_categories.csv  shops.csv                   test.gsheet
items.csv            submission.csv
sales_train_v2.csv   submission_yearly_only.csv


In [0]:
def fast_parsedate(s):
    dates = {date: pd.to_datetime(date, dayfirst=True) for date in s.unique()}
    return s.map(dates)
Validate = True

In [0]:
print('Loading Data...')

salesData = pd.read_csv('sales_train_v2.csv')
if Validate:
    TotalSize = salesData.shape[0]
    print(TotalSize)
    ValidationSize = int(5/100 * TotalSize)
    print("Validation Size = ", ValidationSize)
    testData = salesData[0:ValidationSize]
    testData = testData[['shop_id','item_id']]
    testData['ID']= range(1,len(testData)+1)
#     print(testData)
#     print(salesData)
else:
    testData = pd.read_csv('test.csv')
print(testData.shape)
print('Finished Loading')

Loading Data...
2935849
Validation Size =  146792
(146792, 3)
Finished Loading


In [0]:
print('Parsing Data...')

salesData['date'] = fast_parsedate(salesData['date'])

print('Finished Parsing')



Parsing Data...
Finished Parsing


In [0]:
print('Grouping Data...')




if Validate :
    dates=pd.date_range(start = '2013-01-01',end='2015-09-01', freq = 'MS')
else:
    dates=pd.date_range(start = '2013-01-01',end='2015-10-01', freq = 'MS')

    
sales_piv= salesData.pivot_table(index=['item_id','shop_id'], columns='date_block_num',
                                 values='item_cnt_day',aggfunc=np.sum,fill_value=0).reset_index()
print("sales_piv shape",sales_piv.shape)
dates = dates.to_frame(index=False)

Data = pd.merge(testData, sales_piv, on=['item_id','shop_id'], how='left').fillna(0)
Data = Data.drop(labels=['ID', 'shop_id', 'item_id'], axis=1)
if Validate:
    ActualValues = Data.iloc[0:ValidationSize,-1]
    Data = Data.iloc[:, :-1]
    
print(testData.shape)
print(Data.shape)
Data_train_val = Data.values
print(Data_train_val.shape)
print('Finished Grouping')

Grouping Data...
sales_piv shape (424124, 36)
(146792, 3)
(146792, 33)
(146792, 33)
Finished Grouping


In [101]:
print('Clustering data...')
k = 300
print('Using K =', k)

kmeanModel = KMeans(n_clusters=k)
clusters = kmeanModel.fit_predict(Data_train_val)
centroids = kmeanModel.cluster_centers_
print(centroids.shape[1])
print('Finished Clustering')

Clustering data...
Using K = 300
33
Finished Clustering


In [102]:
print(Data_train_val.shape)
print(clusters.shape[0])

(146792, 33)
146792


In [103]:
print('Creating Forecast Models...')

forecastsDict = {}
for model_id in range(centroids.shape[0]):
    
    # take the date-column and the col to be forecasted
    modelToForecast = pd.concat([dates, pd.DataFrame(centroids[model_id])], axis = 1)
    modelToForecast.columns = ['ds', 'y']
    
    growth = 'linear'
    m = Prophet(growth, yearly_seasonality=True)
    m.fit(modelToForecast)

    future = m.make_future_dataframe(periods = 1, freq = 'MS')
    forecastsDict[model_id] = m.predict(future)
#     m.plot(forecastsDict[model_id],
#               uncertainty=True)




INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Creating Forecast Models...


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [104]:
test_predictions = []
checkpoint = 0
print(len(test_predictions))
print(clusters.shape[0])
for test_case_id in range(clusters.shape[0]):
    t_clust = clusters[test_case_id]
    res = forecastsDict[t_clust][['yhat']].values[-1][0]
#     print("res",res)
    test_predictions.append(res)
    checkpoint += 1
    if checkpoint % 5000 == 0:
        print('Checkpoint %d reached'%checkpoint)
        
print(len(test_predictions))        
test_predictions = np.array(test_predictions)
print(test_predictions.shape)

0
146792
Checkpoint 5000 reached
Checkpoint 10000 reached
Checkpoint 15000 reached
Checkpoint 20000 reached
Checkpoint 25000 reached
Checkpoint 30000 reached
Checkpoint 35000 reached
Checkpoint 40000 reached
Checkpoint 45000 reached
Checkpoint 50000 reached
Checkpoint 55000 reached
Checkpoint 60000 reached
Checkpoint 65000 reached
Checkpoint 70000 reached
Checkpoint 75000 reached
Checkpoint 80000 reached
Checkpoint 85000 reached
Checkpoint 90000 reached
Checkpoint 95000 reached
Checkpoint 100000 reached
Checkpoint 105000 reached
Checkpoint 110000 reached
Checkpoint 115000 reached
Checkpoint 120000 reached
Checkpoint 125000 reached
Checkpoint 130000 reached
Checkpoint 135000 reached
Checkpoint 140000 reached
Checkpoint 145000 reached
146792
(146792,)


In [105]:
print(np.array(test_predictions.shape))

[146792]


In [137]:
# np.unique(test_predictions)
print(np.column_stack((np.rint(test_predictions),np.rint(ActualValues))))
acc = np.abs((np.rint(test_predictions)-np.rint(ActualValues)))
acc[acc<50]=0
acc =  np.sum(acc)
acc= 1-acc/np.array(test_predictions).shape[0]
acc = 100* acc
print("Accuracy = ",acc)
# print(accuracy)

[[-0.  0.]
 [-0.  0.]
 [-0.  0.]
 ...
 [-4.  0.]
 [-1.  0.]
 [-1.  0.]]
Accuracy =  94.09708975965992


In [131]:
from sklearn.metrics import mean_squared_error
from math import sqrt
mse = mean_squared_error(ActualValues, test_predictions)
rmse = sqrt(mse)
print('RMSE: %f' % rmse)

RMSE: 8.173551


In [132]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(ActualValues, test_predictions)
print('MAE: %f' % mae)

MAE: 1.119678


In [0]:
test_predictions = np.around(test_predictions, decimals=2)

In [0]:
np.savetxt("submission_yearly_only.csv", test_predictions, delimiter=",")

In [0]:
K = range(5,100)
KM = [KMeans(n_clusters=k).fit(Data_train_val) for k in K]
centroids = [k.cluster_centers_ for k in KM]

D_k = [cdist(Data_train_val, cent, 'euclidean') for cent in centroids]
cIdx = [np.argmin(D,axis=1) for D in D_k]
dist = [np.min(D,axis=1) for D in D_k]
avgWithinSS = [sum(d)/Data_train_val.shape[0] for d in dist]





In [0]:
kIdx = 8
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, avgWithinSS, 'b*-')
ax.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12, 
markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elbow for KMeans clustering')
plt.show()
